In [1]:
import zipfile
import requests
import io
from pathlib import Path

In [34]:
url="https://raw.githubusercontent.com/Chando0185/movie_recommender_system/main/dataset.csv"
movies=pd.read_csv(url)
movies.head()



,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [35]:
movies.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [38]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [39]:
movies.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


### Formatting

### Feature Selection

In [42]:
movies_features=movies[["id","title","genre","overview","popularity"]]
movies_features.head()

,id,title,genre,overview,popularity
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,94.075
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",25.408
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",90.585
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,44.761
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...,57.749


In [48]:
movies_features["tags"]=movies["genre"] + movies["overview"]
movies_features.tags

<ipython-input-48-09efc98de595>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_features["tags"]=movies["genre"] + movies["overview"]


0       Drama,CrimeFramed in the 1940s for the double ...
1       Comedy,Drama,RomanceRaj is a rich, carefree, h...
2       Drama,CrimeSpanning the years 1945 to 1955, a ...
3       Drama,History,WarThe true story of how busines...
4       Drama,CrimeIn the continuing saga of the Corle...
                              ...                        
9995    Action,Adventure,FantasyThe story follows the ...
9996    Action,TV Movie,Science Fiction,Comedy,Adventu...
9997    Action,Science Fiction,WarDuring World War II,...
9998    Adventure,Fantasy,Action,DramaA man named Farm...
9999    Thriller,Action,CrimeSeeking justice for his p...
Name: tags, Length: 10000, dtype: object

In [49]:
movies_tags=movies_features.drop(columns=["overview","genre"])
movies_tags.head()

,id,title,popularity,tags
0,278,The Shawshank Redemption,94.075,"Drama,CrimeFramed in the 1940s for the double ..."
1,19404,Dilwale Dulhania Le Jayenge,25.408,"Comedy,Drama,RomanceRaj is a rich, carefree, h..."
2,238,The Godfather,90.585,"Drama,CrimeSpanning the years 1945 to 1955, a ..."
3,424,Schindler's List,44.761,"Drama,History,WarThe true story of how busines..."
4,240,The Godfather: Part II,57.749,"Drama,CrimeIn the continuing saga of the Corle..."


In [55]:
movies_tags.shape

(10000, 4)

### Numeric Represntation of the data

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(use_idf=True,max_features=1000,stop_words=("english"))
data=tfidf.fit_transform(movies_tags["tags"].values.astype("U")) # 'U' => convet to text data
mtx_features=data.toarray()
mtx_features
mtx_features.shape

(10000, 1000)

In [53]:
vector_features=mtx_features.flatten()
vector_features
vector_features.shape

(10000000,)

### Cosine Similarity

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

sim_mtx=cosine_similarity(mtx_features)
sim_mtx.shape

(10000, 10000)

#### Obtaining the similarity vector for one movie


In [62]:
movies_tags[movies_tags["title"]=="The Godfather"].index[0]

2

In [63]:
sim_mtx[2,:]

array([0.03009812, 0.06114679, 1.        , ..., 0.03713768, 0.05453981,
       0.        ])

> **NOTE**: In index 2 we obtain a perfect similarity (1) because the  mtx is compared the gie movie with itself, thath means the index in the mtx correspond to the same movie => (God Fathe) in the original DataFrame,so we can use the index in the vector to find the top 'N' movies that refers to a given movie

In [65]:
sim_mtx[2].shape

(10000,)

#### Showing  the similarity score for the god Father respect the rest of the movies

In [66]:
list(enumerate(sim_mtx[2]))

[(0, 0.030098122126808312),
 (1, 0.06114678963809231),
 (2, 1.0),
 (3, 0.01989897463528658),
 (4, 0.3164153320750752),
 (5, 0.05748308726322163),
 (6, 0.10660229208160267),
 (7, 0.022676649021933684),
 (8, 0.0),
 (9, 0.0),
 (10, 0.07542446000072796),
 (11, 0.0),
 (12, 0.08613494500242198),
 (13, 0.012159329917936997),
 (14, 0.0),
 (15, 0.11218415916361146),
 (16, 0.05427877552756959),
 (17, 0.05811830401670714),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0409405927722287),
 (21, 0.020877738163514663),
 (22, 0.0),
 (23, 0.05561456828933062),
 (24, 0.027411137855562456),
 (25, 0.17231029281844734),
 (26, 0.09212500331106242),
 (27, 0.051587390498224174),
 (28, 0.037629739364258875),
 (29, 0.01573100347884846),
 (30, 0.0195046518232474),
 (31, 0.01281975872053548),
 (32, 0.0),
 (33, 0.0),
 (34, 0.018423225402461853),
 (35, 0.03210897054902766),
 (36, 0.04909299998278069),
 (37, 0.06859256456757784),
 (38, 0.011733632598034173),
 (39, 0.0),
 (40, 0.0),
 (41, 0.015676617017918937),
 (42, 0.0),
 (43, 0

In [78]:
sorted(list(enumerate(sim_mtx[2])),reverse=True,key=lambda x:x[1])

[(2, 1.0),
 (4569, 0.35201819518822347),
 (4, 0.3164153320750752),
 (194, 0.31512999936546926),
 (4449, 0.3110032989505232),
 (1816, 0.30510523256188476),
 (1773, 0.30486004557262397),
 (6587, 0.29923345840012305),
 (2272, 0.29560932887416214),
 (4296, 0.2885345042908204),
 (9731, 0.2882220091937611),
 (250, 0.2873867288926105),
 (6842, 0.2855336739355364),
 (1233, 0.27969679422964133),
 (330, 0.2673475749016213),
 (897, 0.2530687389990198),
 (7985, 0.25069771455802),
 (8756, 0.2503450466416037),
 (6964, 0.24616348842418626),
 (434, 0.24157721625696765),
 (8086, 0.24015607221826965),
 (5355, 0.23761046400330144),
 (214, 0.23743343782703008),
 (7453, 0.23583056369127217),
 (1354, 0.2354834030959937),
 (5502, 0.23500814140554685),
 (5954, 0.23447734651807356),
 (892, 0.23432748434722195),
 (5625, 0.23404369456643484),
 (2168, 0.23402891615784255),
 (572, 0.23348548877411782),
 (6023, 0.23328808985762006),
 (5010, 0.23272162175159536),
 (9446, 0.23184413096146234),
 (1535, 0.2295180878896

> **Note** : El argumento **key** dentro de la función sorted() **se utiliza para especificar una función que se aplicará a cada elemento de la lista antes de realizar la clasificación**. En este caso, cada elemento es una tupla que contiene dos parte, la primera es el indice del vector de similatidad, y el segundo valor es el valor de similaridad para, **si especificamos x[0] el ordenamiento se hara sobre los indices, si especificamos x[1] el ordenamiento se hara sobre los valores de similaridad.**

### Showing the top 5 movies more like to the good father

In [88]:
# obtain the idx of the movie
idx_movie=movies_tags[movies_tags["title"]=="The Godfather"].index[0]
# use the index in the mtx similarity to find its corresponding similarity score rescpect the rest of the movies and itself
distance=sorted(list(enumerate(sim_mtx[idx_movie])),reverse=True,key=lambda x:x[1])

# Showing the top 5 fives more related to the selected movie
for (idx,similarity_score) in distance[0:6]:
    if idx != 2:
      print(movies_tags.iloc[idx]["title"])

House of Gucci
The Godfather: Part II
Michael Jackson's Thriller
Stealing Beauty
Road to Perdition


### Defining a function that return a list of movies


In [94]:
def recommendation_movies(movie_name:str,n_movies:int):

  idx_movie=movies_tags[movies_tags["title"]==movie_name].index[0]

  distance=sorted(list(enumerate(sim_mtx[idx_movie])),reverse=True,key=lambda x:x[1])

  for (idx,similarity_score) in distance[:n_movies+1]:

    if idx !=idx_movie:
      print(movies_tags.iloc[idx].title)


In [95]:
recommendation_movies("The Godfather",n_movies=5)

House of Gucci
The Godfather: Part II
Michael Jackson's Thriller
Stealing Beauty
Road to Perdition


### Saving movie data and similarity scores

In [96]:
import pickle
pickle.dump(movies_tags,open("movies_list.pkl","wb"))
pickle.dump(sim_mtx,open("similarity_matrix.pkl","wb"))


In [98]:
pickle.load(open("movies_list.pkl","rb"))

,id,title,popularity,tags
0,278,The Shawshank Redemption,94.075,"Drama,CrimeFramed in the 1940s for the double ..."
1,19404,Dilwale Dulhania Le Jayenge,25.408,"Comedy,Drama,RomanceRaj is a rich, carefree, h..."
2,238,The Godfather,90.585,"Drama,CrimeSpanning the years 1945 to 1955, a ..."
3,424,Schindler's List,44.761,"Drama,History,WarThe true story of how busines..."
4,240,The Godfather: Part II,57.749,"Drama,CrimeIn the continuing saga of the Corle..."
...,...,...,...,...
9995,10196,The Last Airbender,98.322,"Action,Adventure,FantasyThe story follows the ..."
9996,331446,Sharknado 3: Oh Hell No!,12.490,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,18.333,"Action,Science Fiction,WarDuring World War II,..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,15.159,"Adventure,Fantasy,Action,DramaA man named Farm..."
